In [1]:
from pyspark.sql import SparkSession

# Configuration de la SparkSession pour se connecter au cluster
spark = SparkSession.builder \
    .master("spark://pyspark-master:7077") \
    .appName("MySparkApp") \
    .config("spark.executor.memory", "1g") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.6.0.jar") \
    .getOrCreate()

In [2]:
df_spark = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://pg-ds-dellstore:5432/dellstore") \
    .option("dbtable", "public.customer_orders") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

In [3]:
#df_spark.show()
# Montrer un échantillon des données
df_spark.show(5)

# Exemple 1 : Filtrer les clients ayant un revenu supérieur à 50 000
df_high_income = df_spark.filter(df_spark["income"] > 50000)

# Montrer les résultats filtrés
df_high_income.show()

# Exemple 2 : Calculer la moyenne des montants totaux par pays
df_avg_total_by_country = df_spark.groupBy("country").avg("totalamount")

# Renommer les colonnes pour plus de clarté
df_avg_total_by_country = df_avg_total_by_country.withColumnRenamed("avg(totalamount)", "avg_totalamount")

# Montrer les moyennes par pays
df_avg_total_by_country.show()

+----------+---------+----------+---------+-------+-----+-----+------------+---+------+------+-------+----------+---------+-----+-----------+
|customerid|firstname|  lastname| username|   city|state|  zip|     country|age|gender|income|orderid| orderdate|netamount|  tax|totalamount|
+----------+---------+----------+---------+-------+-----+-----+------------+---+------+------+-------+----------+---------+-----+-----------+
|      7888|   RNGILZ|JDZTEGAEMA| user7888|BRLLCUQ|   NH|96140|          US| 49|     F| 20000|      1|2004-01-27|   313.24|25.84|     339.08|
|      4858|   SKULRB|OVWOIYIDDL| user4858|IQIDDJY|   IA|98082|          US| 56|     M| 20000|      2|2004-01-01|    54.90| 4.53|      59.43|
|     15399|   YMIUWA|QWMSNYIQOS|user15399|ZQSKLUS| NULL|    0|South Africa| 60|     M| 40000|      3|2004-01-17|   160.10|13.21|     173.31|
|     17019|   CMLJDQ|BAFBVWATJO|user17019|TVSVKVW| NULL|    0|South Africa| 44|     M| 60000|      4|2004-01-28|   106.67| 8.80|     115.47|
|     

In [4]:
# Créer une nouvelle table avec les clients ayant un revenu > 50 000
df_high_income.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://pg-ds-dellstore:5432/dellstore") \
    .option("dbtable", "public.high_income_customers") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [5]:
# Créer une nouvelle table avec la moyenne des montants totaux par pays
df_avg_total_by_country.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://pg-ds-dellstore:5432/dellstore") \
    .option("dbtable", "public.avg_total_by_country") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()